In [8]:
import os
import pandas as pd
import requests
from scipy.interpolate import interp1d


In [9]:
#excel file online
#https://www.fhlbdm.com/webres/File/fixed-rates-2000-present/FHLBDM_Historical_Rates.xlsx
#get all data into dataframe

url = 'https://www.fhlbdm.com/webres/File/fixed-rates-2000-present/FHLBDM_Historical_Rates.xlsx'
r = requests.get(url)
open('temp.xls', 'wb').write(r.content)
df = pd.read_excel('temp.xls')
# df.head()aa

In [10]:
#create dataframe from last row
df_rates=pd.DataFrame(df.iloc[-1,:])
df_rates=df_rates.reset_index()
df_rates.columns=['period','rate']

#drop 'Historical Rate Indications','Overnight', Week 1, 2, and 3 rows
df_rates.drop([0,1,2,3,4], axis=0, inplace=True)
# df_rates

In [11]:
#String to month number dictionary
period_keys=['1 Month', '2 Month', '3 Month', '4 Month', '5 Month', '6 Month', '7 Month', '8 Month', '9 Month', '10 Month', '11 Month', '1 Year', '1 1/2 Year', '2 Year', '3 Year', '4 Year', '5 Year', '6 Year', '7 Year', '8 Year', '9 Year', '10 Year', '11 Year', '12 Year', '13 Year', '14 Year', '15 Year', '16 Year', '17 Year', '18 Year', '19 Year', '20 Year', '25 Year', '30 Year']
period_values=[1,2,3,4,5,6,7,8,9,10,11,12,18,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240,252,264]
time_dict =  dict(zip(period_keys, period_values))

In [12]:
#Remove weeks from df_rates dataframe (just in case)
df_rates=df_rates[(df_rates.period.str.contains('Year') | df_rates.period.str.contains('Month'))]
# df_rates

In [13]:
#replace 'Month 1' string by month number (as example), to all months and years
df_rates=df_rates.replace({"period": time_dict})

#create period and rate dictionary from dataframe
rates_dict = dict(zip(df_rates.period,df_rates.rate))
# rates_dict

In [49]:
#Create new dataframe with 360 months
df_cof =pd.DataFrame( columns=['Month' , 'FHLBDM_Rate'])
df_cof.Month = list(range(1,361))
#Replace month rates with rates dictionary
df_cof.FHLBDM_Rate = df_cof.Month.map(rates_dict)
df_cof

,Month,FHLBDM_Rate
0,1,1.94
1,2,2.03
2,3,2.12
3,4,2.36
4,5,2.52
...,...,...
355,356,NaN
356,357,NaN
357,358,NaN
358,359,NaN


In [50]:
#feed the interpolate function only real numbers
f = interp1d(df_rates.period, df_rates.rate, kind='linear', fill_value="extrapolate")

In [52]:
df_cof.FHLBDM_Rate = f(df_cof.Month)
df_cof

,Month,FHLBDM_Rate
0,1,1.940000
1,2,2.030000
2,3,2.120000
3,4,2.360000
4,5,2.520000
...,...,...
355,356,4.716667
356,357,4.720000
357,358,4.723333
358,359,4.726667


### CREDIT CAPITAL 
Calculate credit capital interpolation table from risk rating
1. Hardcode (A) risk rating.  Currently using a 1 to 5 scale.  This should be flexible to accomodate different ratings scales (e.g., A to F, 1 to 10)
2. Hardcode a set of 2 dimmensional tables (Risk vs Time) (A)'PD-LGD' (B)'Probability of Default', (C)'Cumulative Provision for Credit Loss', and (D) 'Guarantee Factor' table.  Column headers are risk rating.  Rows are term in months

In [279]:
#imports

from math import exp, sqrt
from scipy.stats import norm

#harcode 2 dimmensional tables

#(A) PD-LGD
df_risk_rating = pd.DataFrame(data={'risk_rating':[1,2,3,4,5],'annual_loan_loss':[0.0002,0.0005,0.0025,0.0065,0.0105],'LGD':[0.335,0.34,0.345,0.35,0.37]})

#(B) Probability of Default.  Numbers in percentage (%).  Devide by 100 to convert to numbers
prob_default_data ={
    'Month': [12,24,36,48,60,72,84,96,108,120],
    '1':[0.02,0.03,0.03,0.05,0.06,0.08,0.1,0.12,0.14,0.15],
    '2':[0.05,0.08,0.11,0.15,0.18,0.21,0.23,0.26,0.28,0.29],
    '3':[0.25,0.26,0.27,0.3,0.32,0.34,0.35,0.37,0.38,0.39],
    '4':[0.65,0.65,0.65,0.65,0.66,0.66,0.67,0.67,0.68,0.68],
    '5':[1.05,1.15,1.44,1.7,1.89,1.99,2.03,2.03,2,1.95],
}

df_prob_default = pd.DataFrame(data=prob_default_data)
df_prob_default.set_index('Month' ,inplace=True)
df_prob_default=df_prob_default/100

#(C) Cummulative Provision for Credit Loss.   Numbers in percentage (%).  Devide by 100 to convert to numbers
credit_loss_data={
     'Month': [12,24,36,48,60,72,84,96,108,120],
    '1':[0.02,0.05,0.1,0.19,0.31,0.48,0.68,0.93,1.22,1.53],
    '2':[0.05,0.15,0.33,0.59,0.9,1.25,1.64,2.05,2.49,2.94],
    '3':[0.25,0.51,0.82,1.19,1.59,2.02,2.47,2.93,3.41,3.89],
    '4':[0.05,0.15,0.33,0.59,0.9,1.25,1.64,2.05,2.49,2.94],
    '5':[1.05,2.31,4.33,6.81,9.43,11.94,14.22,16.24,17.99,19.51],
}

df_credit_loss = pd.DataFrame(data=credit_loss_data)
df_credit_loss.set_index('Month' ,inplace=True)
df_credit_loss = df_credit_loss/100

#(D) Guarantee Factors.   Numbers in percentage (%).  Devide by 100 to convert to numbers
guarantee_factor_data={
     'Month': [1,12,60,120],
    '1':[70,70,80,100],
    '2':[70,70,80,100],
    '3':[70,70,80,100],
    '4':[70,70,80,100],
    '5':[70,70,80,100],
}
df_guarantee_factor = pd.DataFrame(data=guarantee_factor_data)
df_guarantee_factor.set_index('Month' ,inplace=True)
df_guarantee_factor=df_guarantee_factor/100

df_guarantee_factor


,1,2,3,4,5
Month,,,,,
1,0.7,0.7,0.7,0.7,0.7
12,0.7,0.7,0.7,0.7,0.7
60,0.8,0.8,0.8,0.8,0.8
120,1.0,1.0,1.0,1.0,1.0


In [280]:
#create credit capital data frame
df_credit_capital = pd.DataFrame(data={
    'Credit Capital Index': ['risk_rating','probability_default','annual_pcl','asset_corr','confidence','capital'],
    '12':"",
    '60':"",
    '120':""
})

#populate credit capital data frame
df_credit_capital.set_index("Credit Capital Index",inplace = True)
#assume risk rating
df_credit_capital.loc['risk_rating']=2
#get PD
df_credit_capital.loc['probability_default']=list(df_prob_default.loc[[12,60,120],'2'])
#get annual pcl
df_credit_capital.loc['annual_pcl']=list(df_credit_loss.loc[[12,60,120],'2'])
#assume confidence level.  99.9% is a good number (trial and error).  This hardcoded assumption does NOT change
df_credit_capital.loc['confidence']=0.999
df_credit_capital


,12,60,120
Credit Capital Index,,,
risk_rating,2,2,2
probability_default,0.0005,0.0018,0.0029
annual_pcl,0.0005,0.009,0.0294
asset_corr,,,
confidence,0.999,0.999,0.999
capital,,,


In [281]:
#Rho https://www.bis.org/bcbs/irbriskweight.pdf
#Correlation (R) = 0.12 × (1 – EXP (-50 × PD)) / (1 - EXP(-50))
#+ 0.24 × [1 - (1 - EXP(-50 × PD))/(1 - EXP(-50))]
#– 0.04 × (1 – (S-5)/45)  ---> we don't do this size adjustment
# page 13 in PDF

def rhoCalc(pdef):
    return 0.12*(1-exp(-50*pdef))/(1-exp(-50))  +  0.24*(1-(1-exp(-50*pdef))/(1-exp(-50)))  


#Credit Capital (%) K 
# K =1.06*(NORMSDIST(NORMSINV(pcl)*SQRT(1/(1-rho))+NORMSINV(confidence)*SQRT(rho/(1-rho)))-pcl)

# https://support.precisionlender.com/hc/en-us/articles/360042822873-How-to-Determine-Capital-Calculation#:~:text=PrecisionLender's%20pricing%20solution%20is%20a,by%20the%20loan%20or%20deposit.
# why multiply by 1.06?  Well -> https://www.bis.org/publ/bcbs128b.pdf
# "The current best estimate of the scaling factor is 1.06. National authorities will continue to monitor capital
# requirements during the implementation period of this Framework. Moreover, the Committee will monitor
# national experiences with this Framework."
# https://www.lw.com/presentations/regulatory-capital-requirements-for-european-banks

#normal distribution -> norm.cdf(0.95)
#inverse normal distribution ->  norm.ppf(0.95)

def capitalCalc(values_list):
    pcl, rho, confidence = values_list
    return 1.06*(norm.cdf(norm.ppf(pcl)*sqrt(1/(1-rho)) + norm.ppf(confidence)*sqrt(rho/(1-rho)))-pcl)

df_credit_capital.loc['asset_corr']=df_credit_capital.loc['probability_default'].map(rhoCalc)
df_credit_capital.loc['capital'] = df_credit_capital.loc[['annual_pcl','asset_corr','confidence']].apply(capitalCalc,axis=0)
df_credit_capital


,12,60,120
Credit Capital Index,,,
risk_rating,2,2,2
probability_default,0.0005,0.0018,0.0029
annual_pcl,0.0005,0.009,0.0294
asset_corr,0.237037,0.229672,0.223803
confidence,0.999,0.999,0.999
capital,0.021139,0.156606,0.301304


In [284]:
#function inputs Credit Capital dataframe, Guarantee Factor dataframe  
#function returns interpolation dataframe with credit capital(%) K, Loan Loss, and Guarantee for 120 months


def interpolationCalc(df_credit_capital,df_guarantee_factor):
    #Create interpolation table
    df_cc_interpolation =pd.DataFrame( columns=['month' , 'capital','loan_loss','guarantee'])
    df_cc_interpolation.month = list(range(1,121))
    
    x= df_credit_capital.columns.astype(int).insert(0,1)
    y_series = df_credit_capital.loc['capital']
    y = pd.Series(y_series[0]).append(y_series)
    f = interp1d(x, y ,kind='linear', fill_value="extrapolate")
    df_cc_interpolation['capital'] = f(df_cc_interpolation.month)
    
    y_series = df_credit_capital.loc['probability_default']
    y = pd.Series(y_series[0]).append(y_series)
    f = interp1d(x, y ,kind='linear', fill_value="extrapolate")
    df_cc_interpolation['loan_loss'] = f(df_cc_interpolation.month)
    
    risk_rating = df_credit_capital.loc['risk_rating'][0]
    y=df_guarantee_factor.loc[:,str(risk_rating)]
    f = interp1d(x, y ,kind='linear', fill_value="extrapolate")
    df_cc_interpolation['guarantee'] = f(df_cc_interpolation.month)
    
    return df_cc_interpolation


In [285]:
interpolationCalc(df_credit_capital,df_guarantee_factor)
# 

,month,capital,loan_loss,guarantee
0,1,0.021139,0.000500,0.700000
1,2,0.021139,0.000500,0.700000
2,3,0.021139,0.000500,0.700000
3,4,0.021139,0.000500,0.700000
4,5,0.021139,0.000500,0.700000
...,...,...,...,...
115,116,0.291657,0.002827,0.986667
116,117,0.294069,0.002845,0.990000
117,118,0.296481,0.002863,0.993333
118,119,0.298892,0.002882,0.996667
